In [19]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [20]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

(467,)


In [21]:
inputs = []
max_length = 0
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=0)
    a = np.log(a)
    inputs.append(a)

In [22]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [23]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [24]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [25]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 467]), torch.Size([141, 53, 200]), torch.Size([141]))

# 0.6170212765957447 23轮

In [26]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 53, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=53,    # n_filters
                kernel_size=(1,200),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=(53,1)),    # 在 2x2 空间里向下采样, output shape (16, 53, 1)
        )
        self.output = nn.Linear(53, 7)   # fully connected layer, output 10 classes

    def forward(self, x):
        torch.set_default_dtype(torch.double)
        x = x.reshape((-1,1,53,200)).double()
        x = self.conv1(x.double())
        x = x.view(x.size(0), -1)   # 展平
#         x = self.out(x)
        output = self.output(x)
        return output

In [27]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = CNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
start = time.time()
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss} time: {time.time() - start}")

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 53, kernel_size=(1, 200), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=53, out_features=7, bias=True)
)
1: accuracy:0.2198581560283688 loss: 1.9037755804325567 time: 0.2097330093383789
2: accuracy:0.4397163120567376 loss: 1.549300305141994 time: 0.41186094284057617
3: accuracy:0.5177304964539007 loss: 1.2839013002986361 time: 0.5966610908508301
4: accuracy:0.5602836879432624 loss: 0.9303090941539431 time: 0.7908508777618408
5: accuracy:0.5673758865248227 loss: 0.7797673715684522 time: 1.0172820091247559
6: accuracy:0.5460992907801419 loss: 0.47803366497188804 time: 1.2342960834503174
7: accuracy:0.5886524822695035 loss: 0.357710195641941 time: 1.4224300384521484
8: accuracy:0.5957446808510638 loss: 0.29129630251229527 time: 1.6051998138427734
9: accuracy:0.5815602836879432 loss: 0.16800746455333423 time: 1.7896881103515625
1

96: accuracy:0.6099290780141844 loss: 0.0006996444944390921 time: 18.249208211898804
97: accuracy:0.6099290780141844 loss: 0.0006707171280759309 time: 18.413047075271606
98: accuracy:0.6099290780141844 loss: 0.000657162730689593 time: 18.607497930526733
99: accuracy:0.6099290780141844 loss: 0.0005525695155582686 time: 18.80419421195984
100: accuracy:0.6099290780141844 loss: 0.000656409127055443 time: 18.99507784843445
101: accuracy:0.6099290780141844 loss: 0.0006910478282396327 time: 19.18441915512085
102: accuracy:0.6099290780141844 loss: 0.0005858998460414964 time: 19.349805116653442
103: accuracy:0.6099290780141844 loss: 0.0005914224141634275 time: 19.521262884140015
104: accuracy:0.6099290780141844 loss: 0.0005388525933275799 time: 19.688402891159058
105: accuracy:0.6099290780141844 loss: 0.0005770309750560936 time: 19.86892294883728
106: accuracy:0.6099290780141844 loss: 0.0005366708817185436 time: 20.09344220161438
107: accuracy:0.6099290780141844 loss: 0.0005428538133112821 time

192: accuracy:0.6028368794326241 loss: 0.00020562667626083147 time: 35.931596994400024
193: accuracy:0.6028368794326241 loss: 0.00019052557570703295 time: 36.13655209541321
194: accuracy:0.6028368794326241 loss: 0.0002107609963320378 time: 36.33964800834656
195: accuracy:0.6028368794326241 loss: 0.00019450586827836705 time: 36.545604944229126
196: accuracy:0.6028368794326241 loss: 0.0001975190969800446 time: 36.75263500213623
197: accuracy:0.6028368794326241 loss: 0.0001988417631102151 time: 36.96147584915161
198: accuracy:0.6028368794326241 loss: 0.00018871846322655118 time: 37.17354083061218
199: accuracy:0.6028368794326241 loss: 0.00020135617760203166 time: 37.37429690361023
200: accuracy:0.6099290780141844 loss: 0.00021362233706746027 time: 37.57643699645996
